# Assignment 2 Deepankar Varma 102003431 

### Step 1: Import the required libraries
We will need the following libraries to perform face recognition:

OpenCV (cv2) for face detection and face alignment. <br>
dlib for facial landmarks detection.
NumPy for numerical operations.

In [13]:
import cv2
import dlib
import numpy as np


### Step 2: Load the dataset into a list

In [14]:
import os

image_folder = 'Images/Humans'
images = []
for filename in os.listdir(image_folder):
    img = cv2.imread(os.path.join(image_folder, filename))
    if img is not None:
        images.append(img)


### Step 3: Face detection 

We will use the Haar Cascade Classifier for face detection. It is a pre-trained classifier that can detect faces in an image.

In [15]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

face_rects = []
for image in images:
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    for (x,y,w,h) in faces:
        face_rects.append((x,y,w,h))


### Step 4: Face Alignment

To perform face recognition, we need to align all the faces so that they have similar landmarks. We can use the dlib library to detect facial landmarks and align the faces.

In [16]:
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

aligned_faces = []
for image, (x,y,w,h) in zip(images, face_rects):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    landmarks = predictor(gray, dlib.rectangle(x,y,x+w,y+h))
    landmarks = np.array([[p.x, p.y] for p in landmarks.parts()])
    
    # calculate the mean point of the two corners of the eyes in the image
    left_eye = np.mean(landmarks[36:42], axis=0)
    right_eye = np.mean(landmarks[42:48], axis=0)

    # calculate the angle between the line connecting the eyes and the horizontal axis
    dy = right_eye[1] - left_eye[1]
    dx = right_eye[0] - left_eye[0]
    angle = np.degrees(np.arctan2(dy, dx)) - 180

    # calculate the scale factor
    desired_left_eye = (0.35, 0.35)
    dist = np.sqrt((dx ** 2) + (dy ** 2))
    desired_dist = (desired_left_eye[1] - desired_left_eye[0]) * image.shape[1]
    scale = desired_dist / dist

    # calculate the rotation matrix
    center = (left_eye + right_eye) // 2
    M = cv2.getRotationMatrix2D(tuple(center), angle, scale)

    # apply the transformation to the image
    tX = image.shape[1] // 2 - center[0]
    tY = image.shape[0] * 0.35 - center[1]
    M[0, 2] += tX
    M[1, 2] += tY

    aligned_face = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]), flags=cv2.INTER_CUBIC)
    aligned_faces.append(aligned_face)


### Step 5: Feature Extraction

To perform feature extraction, we will use a pre-trained deep learning model that can extract facial features. Here, we will use the VGG-Face model, which is a pre-trained CNN model trained on a large dataset of faces. We will use the last fully connected layer (fc7) of the model as the feature vector.

In [17]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling='avg')

features = []
for aligned_face in aligned_faces:
    # resize the image to the input size of the VGG16 model
    input_image = cv2.resize(aligned_face, (224, 224))

    # preprocess the input image
    input_image = preprocess_input(input_image)

    # extract features using the VGG16 model
    feature = model.predict(np.expand_dims(input_image, axis=0))[0]

    features.append(feature)


1/1 [==============================] - 0s 191ms/step


### Step 6: Feature Matching

To perform feature matching, we can use any distance metric to calculate the distance between two feature vectors. Here, we will use the Euclidean distance.

In [18]:
import dlib
import numpy as np

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

def align_face(face):
    gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)
    if len(rects) == 0:
        return None
    shape = predictor(gray, rects[0])
    landmarks = np.array([[p.x, p.y] for p in shape.parts()])
    eye_center = landmarks[36:42].mean(axis=0)
    dx = landmarks[45, 0] - landmarks[36, 0]
    dy = landmarks[45, 1] - landmarks[36, 1]
    angle = np.degrees(np.arctan2(dy, dx)) - 90
    M = cv2.getRotationMatrix2D(tuple(eye_center), angle, 1)
    aligned_face = cv2.warpAffine(face, M, (face.shape[1], face.shape[0]), flags=cv2.INTER_CUBIC)
    return aligned_face


In [22]:
from scipy.spatial.distance import euclidean

def distance(feature1, feature2):
    return euclidean(feature1, feature2)

def match(feature, database, threshold):
    distances = [distance(feature, db_feature) for db_feature in database]
    min_distance = min(distances)
    if min_distance < threshold:
        return np.argmin(distances)
    else:
        return None

database = np.array(features)
threshold = 0.6

# test the model on a new image
test_image = cv2.imread('test1.jpg')
gray = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=10)
for (x,y,w,h) in faces:
    face = test_image[y:y+h, x:x+w]
    aligned_face = align_face(face)
    if aligned_face is not None:
        input_image = cv2.resize(aligned_face, (224, 224))
        input_image = preprocess_input(input_image)
        feature = model.predict(np.expand_dims(input_image, axis=0))[0]
        index = match(feature, database, threshold)
        if index is not None:
            print(f"Matched with image {index}")
        else:
            print("No match found")
    else:
        print("Face alignment failed")


1/1 [==============================] - 0s 178ms/step
No match found
